In [ ]:
pip install transformers

In [11]:
import pandas as pd
data = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/프로젝트/11.17/DATA4.xlsx", engine="openpyxl")

In [17]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# "klue/roberta-small" 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")

# "klue/roberta-small" 모델 로드
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-small", num_labels=max_label_index + 1)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from sklearn.model_selection import train_test_split

# 데이터 셔플링 및 분할
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


In [19]:
# 레이블 처리
import numpy as np

# 레이블 인덱스의 최대값을 정확하게 계산
max_label_index = int(data['label_idx'].str.split(',').explode().apply(int).max())

def create_label_vector(label_str):
    labels = [int(label) for label in label_str.split(',')]
    label_vector = np.zeros(max_label_index + 1)
    label_vector[labels] = 1
    return label_vector

binary_labels = np.array([create_label_vector(label_str) for label_str in data['label_idx']])


In [20]:
import torch

# 손실 함수 설정 (BCEWithLogitsLoss)
loss_function = torch.nn.BCEWithLogitsLoss()

# 옵티마이저 설정 (Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)  # 학습률은 실험적으로 조정 가능

In [21]:
from torch.utils.data import DataLoader, TensorDataset

# 토큰화된 데이터 생성
tokenized_train = tokenizer(list(train_data['SENTENCE']), padding=True, truncation=True, return_tensors="pt")
tokenized_test = tokenizer(list(test_data['SENTENCE']), padding=True, truncation=True, return_tensors="pt")

# 레이블 데이터 생성
train_labels = train_data['label_idx'].apply(create_label_vector).tolist()
test_labels = test_data['label_idx'].apply(create_label_vector).tolist()

# TensorDataset 생성
train_dataset = TensorDataset(tokenized_train['input_ids'], tokenized_train['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(tokenized_test['input_ids'], tokenized_test['attention_mask'], torch.tensor(test_labels))

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)


<ipython-input-21-b3fcb11de4ae>:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  train_dataset = TensorDataset(tokenized_train['input_ids'], tokenized_train['attention_mask'], torch.tensor(train_labels))


In [ ]:
from tqdm import tqdm
import torch

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for batch in tqdm(train_loader, desc=f'Train Epoch: {epoch + 1}'):
        batch_input_ids, batch_attention_mask, batch_labels = batch

        optimizer.zero_grad()

        outputs = model(batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)
        loss = outputs.loss
        total_loss += loss.item()

        # 예측 값 계산 (시그모이드 함수 적용)
        predictions = torch.sigmoid(outputs.logits)
        predicted_labels = (predictions > 0.5).float()  # 임계값 0.5

        # 정확도 계산
        correct_predictions += (predicted_labels == batch_labels).sum().item()
        total_predictions += batch_labels.numel()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss}, Accuracy: {accuracy}")


Train Epoch: 1:  56%|█████▌    | 439/787 [35:40<28:15,  4.87s/it]